<a href="https://colab.research.google.com/github/vanessajen/FlatLay-Generator/blob/main/FlatLay_Generator_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ControlNet Flatten V2 - Auto detect input, full process

# --- Step 0: Install necessary libraries ---
!pip install diffusers transformers accelerate controlnet_aux --upgrade

# --- Step 0.5: Set environment + create folders automatically ---
import os
os.environ["DIFFUSERS_FLAX_AVAILABLE"] = "0"

input_folder = "/content/drive/MyDrive/FlatLay_Input"
output_folder = "/content/drive/MyDrive/FlatLay_Demo_Output"

os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Check for available input images (jpg, png)
input_images = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

if not input_images:
    print(f"⚡️ Chưa có ảnh input. Vui lòng upload ảnh (.jpg/.png) vào: {input_folder}")
else:
    print(f"🔍 Tìm thấy {len(input_images)} ảnh trong {input_folder}: {input_images}")

# --- Step 1: Import Python libraries ---
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from controlnet_aux import CannyDetector
from PIL import Image
import numpy as np

# --- Step 2: Load ControlNet model ---
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

# --- Step 3: Canny Edge Detector ---
canny = CannyDetector()

# --- Step 4: Function to process flat lay ---
def process_flatlay(input_image_path, prompt, output_path):
    image = Image.open(input_image_path)
    image = image.convert("RGB")
    image = image.resize((512, 512))  # Resize về 512x512 cho chuẩn model

    # Create canny edge map
    canny_image = canny(image)

    # Generate output image
    generated_image = pipe(
        prompt,
        num_inference_steps=30,
        guidance_scale=9.0,
        image=canny_image
    ).images[0]

    # Save the result
    generated_image.save(output_path)

# --- Step 5: Run on all input images ---
prompt = "flat lay minimal pastel background, photography, highly detailed, white background"

for img_name in input_images:
    input_path = os.path.join(input_folder, img_name)
    output_name = os.path.splitext(img_name)[0] + "_flatlay.png"
    output_path = os.path.join(output_folder, output_name)

    print(f"\u2728 Đang xử lý: {img_name}")
    process_flatlay(input_path, prompt, output_path)
    print(f"\u2705 Đã lưu ảnh: {output_name}")

print("\n\ud83c\udf3f\u2728 Hoàn tất tạo ảnh Flat Lay!")
